In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

In [2]:
df = pd.read_excel(r"data/performance.xlsx")

In [3]:
df

,Data,Grupo,Detentora,Vendor,eNodeB,Cell,SET_NUMBER,TIM_PRB_AVAIL (Unid),TIM_PRB_UTIL_MEAN_DL (%),TIM_PRB_UTIL_MEAN_DL_DEN (Unid),TIM_PRB_UTIL_MEAN_DL_NUM (Unid),TIM_THROU_USER_PDCP_DL (Kbps),TIM_THROU_USER_PDCP_DL_DEN (ms),TIM_THROU_USER_PDCP_DL_NUM (bit),TIM_USERS_RRC_CONN_MEAN_SUM_ALLOP (Unid),TIM_VOLUME_DADOS_DLUL_ALLOP (KB),TIM_VOLUME_DADOS_DL_ALLOP (KB)
0,2025-09-15 00:00,LMS_UFF_TNE,TIM,HUAWEI,4G-BRBRJ0,4G-BRBRJ0-07-A,1,50.0,4.544,50.0,227.2,4180.205130,383191.0,1.601817e+09,6.865,226689.593750,207852.999023
1,2025-09-15 00:00,LMS_UFF_TNE,TIM,HUAWEI,4G-BRBRJ0,4G-BRBRJ0-07-B,1,50.0,5.398,50.0,269.9,7406.687408,332539.0,2.463012e+09,6.736,371498.003906,333790.555664
2,2025-09-15 00:00,LMS_UFF_TNE,TIM,HUAWEI,4G-BRBRJ0,4G-BRBRJ0-07-C,1,50.0,4.316,50.0,215.8,1499.223530,428735.0,6.427696e+08,5.408,92756.415039,80417.963867
3,2025-09-15 00:00,LMS_UFF_TNE,TIM,HUAWEI,4G-BRBRJ0,4G-BRBRJ0-26-A,1,100.0,0.935,100.0,93.5,6543.122840,3183.0,2.082676e+07,2.225,7556.021484,4790.798828
4,2025-09-15 00:00,LMS_UFF_TNE,TIM,HUAWEI,4G-BRBRJ0,4G-BRBRJ0-26-B,1,100.0,0.892,100.0,89.2,7135.451613,744.0,5.308776e+06,0.199,832.632812,725.502930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20803,2025-09-29 23:00,LMS_UFF_TNE,TIM,HUAWEI,4G-OBOBJ0,4G-OBOBJ0-26-B,0,100.0,3.184,100.0,318.4,39158.936126,25018.0,9.796783e+08,4.355,159533.857422,142852.800781
20804,2025-09-29 23:00,LMS_UFF_TNE,TIM,HUAWEI,4G-OBOBJ0,4G-OBOBJ0-26-C,0,100.0,3.003,100.0,300.3,40296.739137,6191.0,2.494771e+08,1.001,35449.901367,32702.747070
20805,2025-09-29 23:00,LMS_UFF_TNE,TIM,HUAWEI,4G-OBOBJ0,4G-OBOBJ0-26-I,0,50.0,6.616,50.0,330.8,24507.526096,74437.0,1.824267e+09,4.844,348986.671875,324642.206055
20806,2025-09-29 23:00,LMS_UFF_TNE,TIM,HUAWEI,4G-OBOBJ0,4G-OBOBJ0-26-J,0,50.0,6.900,50.0,345.0,43202.575719,67321.0,2.908441e+09,17.281,537333.431641,477212.602539


In [4]:
na_counts = df.isna().sum()
na_counts_df = na_counts.reset_index()
na_counts_df.columns = ['Coluna', 'Qtd_NaNs']
print(na_counts_df)

df = df.dropna()

                                      Coluna  Qtd_NaNs
0                                       Data         0
1                                      Grupo         0
2                                  Detentora         0
3                                     Vendor         0
4                                     eNodeB         0
5                                       Cell         0
6                                 SET_NUMBER         0
7                       TIM_PRB_AVAIL (Unid)        72
8                   TIM_PRB_UTIL_MEAN_DL (%)       186
9            TIM_PRB_UTIL_MEAN_DL_DEN (Unid)        72
10           TIM_PRB_UTIL_MEAN_DL_NUM (Unid)        72
11             TIM_THROU_USER_PDCP_DL (Kbps)       636
12           TIM_THROU_USER_PDCP_DL_DEN (ms)        72
13          TIM_THROU_USER_PDCP_DL_NUM (bit)        72
14  TIM_USERS_RRC_CONN_MEAN_SUM_ALLOP (Unid)        72
15          TIM_VOLUME_DADOS_DLUL_ALLOP (KB)        72
16            TIM_VOLUME_DADOS_DL_ALLOP (KB)        72


In [5]:
df = df.drop(["TIM_PRB_UTIL_MEAN_DL_DEN (Unid)",
"TIM_PRB_UTIL_MEAN_DL_NUM (Unid)",
"TIM_THROU_USER_PDCP_DL_DEN (ms)",
"TIM_THROU_USER_PDCP_DL_NUM (bit)","SET_NUMBER"] ,
 axis =1)

df

,Data,Grupo,Detentora,Vendor,eNodeB,Cell,TIM_PRB_AVAIL (Unid),TIM_PRB_UTIL_MEAN_DL (%),TIM_THROU_USER_PDCP_DL (Kbps),TIM_USERS_RRC_CONN_MEAN_SUM_ALLOP (Unid),TIM_VOLUME_DADOS_DLUL_ALLOP (KB),TIM_VOLUME_DADOS_DL_ALLOP (KB)
0,2025-09-15 00:00,LMS_UFF_TNE,TIM,HUAWEI,4G-BRBRJ0,4G-BRBRJ0-07-A,50.0,4.544,4180.205130,6.865,226689.593750,207852.999023
1,2025-09-15 00:00,LMS_UFF_TNE,TIM,HUAWEI,4G-BRBRJ0,4G-BRBRJ0-07-B,50.0,5.398,7406.687408,6.736,371498.003906,333790.555664
2,2025-09-15 00:00,LMS_UFF_TNE,TIM,HUAWEI,4G-BRBRJ0,4G-BRBRJ0-07-C,50.0,4.316,1499.223530,5.408,92756.415039,80417.963867
3,2025-09-15 00:00,LMS_UFF_TNE,TIM,HUAWEI,4G-BRBRJ0,4G-BRBRJ0-26-A,100.0,0.935,6543.122840,2.225,7556.021484,4790.798828
4,2025-09-15 00:00,LMS_UFF_TNE,TIM,HUAWEI,4G-BRBRJ0,4G-BRBRJ0-26-B,100.0,0.892,7135.451613,0.199,832.632812,725.502930
...,...,...,...,...,...,...,...,...,...,...,...,...
20803,2025-09-29 23:00,LMS_UFF_TNE,TIM,HUAWEI,4G-OBOBJ0,4G-OBOBJ0-26-B,100.0,3.184,39158.936126,4.355,159533.857422,142852.800781
20804,2025-09-29 23:00,LMS_UFF_TNE,TIM,HUAWEI,4G-OBOBJ0,4G-OBOBJ0-26-C,100.0,3.003,40296.739137,1.001,35449.901367,32702.747070
20805,2025-09-29 23:00,LMS_UFF_TNE,TIM,HUAWEI,4G-OBOBJ0,4G-OBOBJ0-26-I,50.0,6.616,24507.526096,4.844,348986.671875,324642.206055
20806,2025-09-29 23:00,LMS_UFF_TNE,TIM,HUAWEI,4G-OBOBJ0,4G-OBOBJ0-26-J,50.0,6.900,43202.575719,17.281,537333.431641,477212.602539


In [6]:
#df["SET_NUMBER"] = df["SET_NUMBER"].astype(str)

In [7]:
df = df.astype({col: int for col in df.select_dtypes('bool').columns})

In [8]:
df

,Data,Grupo,Detentora,Vendor,eNodeB,Cell,TIM_PRB_AVAIL (Unid),TIM_PRB_UTIL_MEAN_DL (%),TIM_THROU_USER_PDCP_DL (Kbps),TIM_USERS_RRC_CONN_MEAN_SUM_ALLOP (Unid),TIM_VOLUME_DADOS_DLUL_ALLOP (KB),TIM_VOLUME_DADOS_DL_ALLOP (KB)
0,2025-09-15 00:00,LMS_UFF_TNE,TIM,HUAWEI,4G-BRBRJ0,4G-BRBRJ0-07-A,50.0,4.544,4180.205130,6.865,226689.593750,207852.999023
1,2025-09-15 00:00,LMS_UFF_TNE,TIM,HUAWEI,4G-BRBRJ0,4G-BRBRJ0-07-B,50.0,5.398,7406.687408,6.736,371498.003906,333790.555664
2,2025-09-15 00:00,LMS_UFF_TNE,TIM,HUAWEI,4G-BRBRJ0,4G-BRBRJ0-07-C,50.0,4.316,1499.223530,5.408,92756.415039,80417.963867
3,2025-09-15 00:00,LMS_UFF_TNE,TIM,HUAWEI,4G-BRBRJ0,4G-BRBRJ0-26-A,100.0,0.935,6543.122840,2.225,7556.021484,4790.798828
4,2025-09-15 00:00,LMS_UFF_TNE,TIM,HUAWEI,4G-BRBRJ0,4G-BRBRJ0-26-B,100.0,0.892,7135.451613,0.199,832.632812,725.502930
...,...,...,...,...,...,...,...,...,...,...,...,...
20803,2025-09-29 23:00,LMS_UFF_TNE,TIM,HUAWEI,4G-OBOBJ0,4G-OBOBJ0-26-B,100.0,3.184,39158.936126,4.355,159533.857422,142852.800781
20804,2025-09-29 23:00,LMS_UFF_TNE,TIM,HUAWEI,4G-OBOBJ0,4G-OBOBJ0-26-C,100.0,3.003,40296.739137,1.001,35449.901367,32702.747070
20805,2025-09-29 23:00,LMS_UFF_TNE,TIM,HUAWEI,4G-OBOBJ0,4G-OBOBJ0-26-I,50.0,6.616,24507.526096,4.844,348986.671875,324642.206055
20806,2025-09-29 23:00,LMS_UFF_TNE,TIM,HUAWEI,4G-OBOBJ0,4G-OBOBJ0-26-J,50.0,6.900,43202.575719,17.281,537333.431641,477212.602539


In [9]:
na_counts = df.isna().sum()
na_counts_df = na_counts.reset_index()
na_counts_df.columns = ['Coluna', 'Qtd_NaNs']
print(na_counts_df)

                                      Coluna  Qtd_NaNs
0                                       Data         0
1                                      Grupo         0
2                                  Detentora         0
3                                     Vendor         0
4                                     eNodeB         0
5                                       Cell         0
6                       TIM_PRB_AVAIL (Unid)         0
7                   TIM_PRB_UTIL_MEAN_DL (%)         0
8              TIM_THROU_USER_PDCP_DL (Kbps)         0
9   TIM_USERS_RRC_CONN_MEAN_SUM_ALLOP (Unid)         0
10          TIM_VOLUME_DADOS_DLUL_ALLOP (KB)         0
11            TIM_VOLUME_DADOS_DL_ALLOP (KB)         0


In [10]:
df.to_csv("data/df_pp.csv", index=False)